<a href="https://colab.research.google.com/github/husainattar/Rail-Rush/blob/master/ExperimentHogCascade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import cv2 
import numpy as np
from __future__ import print_function
from imutils.object_detection import non_max_suppression
from imutils import paths
import argparse
import imutils
import matplotlib.pyplot as plt
%matplotlib inline



In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [0]:
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

In [0]:
def detect(image):
  image = imutils.resize(image, width=min(400, image.shape[1]))
  orig = image.copy()
 	# detect people in the image
  (rects, weights) = hog.detectMultiScale(image, winStride=(4, 4),padding=(8, 8), scale=1.05)
 
	# draw the original bounding boxes
  for (x, y, w, h) in rects:
    cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 0, 255), 2)
  

	# apply non-maxima suppression to the bounding boxes using a
	# fairly large overlap threshold to try to maintain overlapping
	# boxes that are still people
  rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
  pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)

  val=len(pick)
 
	# draw the final bounding boxes
  for (xA, yA, xB, yB) in pick:
    cv2.rectangle(image, (xA, yA), (xB, yB), (0, 255, 0), 2)
  
  return val

In [0]:
#Frame creator function
def getFrame(sec):
    cap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = cap.read()
    if hasFrames:
      img = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
      val=detect(img)

      #To Access the global variable define global variable then assign value to it.

      global minval
      minval=min(val,minval)
      global maxval
      maxval=max(val,maxval)
      
      if sec==3:
        #plt.imshow(image)
        #cv2.imwrite('/content/drive/My Drive/Colab Notebooks/data/'+str(sec)+' sec.jpg', image)
        #cv2.imwrite(str(sec))
        print(val)    
       # save frame as JPG file
    return hasFrames

# Playing video from file:
cap = cv2.VideoCapture('/content/drive/My Drive/Colab Notebooks/train1.mp4')

sec = 0
frameRate = 1
minval=1000000
maxval=0
#it will capture image in each 0.5 second
success = getFrame(sec)
while success:
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)
print("min val:",minval)
print("max val:",maxval)
cap.release()
cv2.destroyAllWindows()